In [ ]:
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

subject_list = [f'sub-{str(i).zfill(2)}' for i in range(1, 11)]

all_list=[]

eeg_list = []
for subject in subject_list:
    data_path= f'/dev/shm/wht/datasets/things-eeg-small/Preprocessed_data_250Hz_whiten/{subject}/test.pt'
    loaded_data = torch.load(data_path)
    loaded_data['eeg'] = torch.from_numpy(loaded_data['eeg'])
    selected_ch = ['P7', 'P5', 'P3', 'P1','Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8','O1', 'Oz', 'O2']
    # selected_ch = None
    channels = ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F5', 'F3',
                            'F1', 'F2', 'F4', 'F6', 'F8', 'FT9', 'FT7', 'FC5', 'FC3', 'FC1', 
                            'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'FT10', 'T7', 'C5', 'C3', 'C1',
                            'Cz', 'C2', 'C4', 'C6', 'T8', 'TP9', 'TP7', 'CP5', 'CP3', 'CP1', 
                            'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'TP10', 'P7', 'P5', 'P3', 'P1',
                            'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8',
                            'O1', 'Oz', 'O2']
    if selected_ch:
        selected_idx = [channels.index(ch) for ch in selected_ch]
        loaded_data['eeg'] = loaded_data['eeg'][:,:,selected_idx]
    else:
        selected_ch = channels
    eeg = loaded_data['eeg'].to(torch.float32)
    num_ch = len(selected_ch) if selected_ch else 63
    eeg_mean = eeg.mean(axis=1)

    eeg_list.append(eeg_mean)

    norm_list =[]
    for i in range(eeg.shape[0]):
        diff_value = eeg[i] - eeg_mean[i]
        norm_v = torch.norm(diff_value, 'fro')
        norm_list.append(norm_v.item())
    
    all_list.append(norm_list)

In [ ]:
data = [ele[:2] for ele in eeg_list]
data = np.array(data)
print(data.shape)
data = data.reshape(10*2*1,-1)

print(data.shape)

In [ ]:
from sklearn.decomposition import PCA
import umap
n_components = 100
n_neighbors = 50
pca = PCA(n_components=n_components)
data_pca = pca.fit_transform(data)
reducer = umap.UMAP(n_neighbors=n_neighbors, random_state=0)
embedding_2d = reducer.fit_transform(data_pca)


In [ ]:
plt.figure(figsize=(8, 8),dpi=800) 
n_subjects = 10 
n_stimuli = 2
n_samples_per_stimulus = 80
subjects = np.repeat([f"Subject {i+1}" for i in range(n_subjects)], n_stimuli * n_samples_per_stimulus)

data_types = np.tile(np.repeat([f"Stimulus {j+1}" for j in range(n_stimuli)], n_samples_per_stimulus), n_subjects)
x = embedding_2d[:,0]
y = embedding_2d[:,1]
df = pd.DataFrame({
    'Subject': subjects,
    'Data Type': data_types,
    'X': x,
    'Y': y
})

# palette = sns.color_palette("hsv", 10) 
markers = ['o', 's', 'D']

for (data_type, marker) in zip(df['Data Type'].unique(), markers):
    subset = df[df['Data Type'] == data_type]
    sns.scatterplot(data=subset, x='X', y='Y', hue='Subject',style='Data Type', markers={data_type: marker}, s=10,legend=False)
    
plt.xlabel('UMAP 1', fontsize=22)
plt.ylabel('UMAP 2', fontsize=22)


plt.tight_layout()
plt.savefig(f'./figures/subject_stimuli.pdf', format='pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12, 8),dpi=800)

all_v = np.array(all_list)

all_v = (all_v -np.min(all_v)+1e-5)/(np.max(all_v )-np.min(all_v )+1e-5)

data_groups = {subject:norm_list for subject,norm_list in zip(subject_list,all_v)}
df = pd.DataFrame(data_groups)
ax = sns.kdeplot(df, shade=True)
plt.xlabel('Variability Value', fontsize=22)
plt.ylabel('Density', fontsize=22)
plt.xticks(fontsize=16) 
plt.yticks(fontsize=16) 
plt.xlim(-0.1, 0.7)
# plt.ylim(0, 0.5)
plt.setp(ax.get_legend().get_texts(), fontsize=25)

plt.tight_layout()
save_path = 'figures/subject_variability.png'
plt.savefig(save_path.replace('.png','.pdf'), format='pdf')
plt.show()

In [5]:
import sys,os,torch
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
device = torch.device('cuda:6' if torch.cuda.is_available() else 'cpu')

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../')))
from base.utils import instantiate_from_config
from base.data import load_data
import numpy as np
config = '/home/wht/multimodal_brain/src/tasks/base/configs/autoencoder_kl_32x32x4.yaml'
config = OmegaConf.load(config)
model = instantiate_from_config(config['model'])
model = model.to(device)

ckpt_pth = '/home/wht/multimodal_brain/src/tasks/1_eeg_pretrain/exp/VAE Pretrain/version_12/checkpoints/epoch=49-step=16200.ckpt'
ckpt= torch.load(ckpt_pth)
model.load_state_dict(ckpt['state_dict'])

data_config = '/home/wht/multimodal_brain/src/tasks/base/configs/pretrain.yaml'
data_config = OmegaConf.load(data_config)
data_config['data']['test_avg'] = False
train_loader, test_loader = load_data(data_config)

making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 17, 62, 62) = 65348 dimensions.
making attention of type 'vanilla' with 512 in_channels


/tmp/ipykernel_3862392/3950674974.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt= torch.load(ckpt_pth)


RuntimeError: Error(s) in loading state_dict for AutoencoderKL:
	Unexpected key(s) in state_dict: "loss.perceptual_loss.scaling_layer.shift", "loss.perceptual_loss.scaling_layer.scale", "loss.perceptual_loss.net.slice1.0.weight", "loss.perceptual_loss.net.slice1.0.bias", "loss.perceptual_loss.net.slice1.2.weight", "loss.perceptual_loss.net.slice1.2.bias", "loss.perceptual_loss.net.slice2.5.weight", "loss.perceptual_loss.net.slice2.5.bias", "loss.perceptual_loss.net.slice2.7.weight", "loss.perceptual_loss.net.slice2.7.bias", "loss.perceptual_loss.net.slice3.10.weight", "loss.perceptual_loss.net.slice3.10.bias", "loss.perceptual_loss.net.slice3.12.weight", "loss.perceptual_loss.net.slice3.12.bias", "loss.perceptual_loss.net.slice3.14.weight", "loss.perceptual_loss.net.slice3.14.bias", "loss.perceptual_loss.net.slice4.17.weight", "loss.perceptual_loss.net.slice4.17.bias", "loss.perceptual_loss.net.slice4.19.weight", "loss.perceptual_loss.net.slice4.19.bias", "loss.perceptual_loss.net.slic

: 

In [ ]:
eeg_latent = []
labels = []
# eegs = []
for i,batch in enumerate(test_loader):
    eeg, label, img, img_features, text, text_features, session, subject = batch
    eeg =eeg.to(device)
    dec, posterior, z = model(eeg)
    # eegs.append(eeg.view(z.shape[0],-1).cpu().detach().numpy())
    eeg_latent.append(z.view(z.shape[0],-1).cpu().detach().numpy())
    labels.append(subject.cpu().detach().numpy())
eeg_latent_v= np.concatenate(eeg_latent,axis=0)
eeg_latent_v = eeg_latent_v.reshape(eeg_latent_v.shape[0],-1)

# eegs = np.concatenate(eegs,axis=0)W
# eegs_np = eegs.reshape(eegs.shape[0],-1)

labels_np = np.array(labels)
labels_np = labels_np.reshape(-1)

print(eeg_latent_v.shape, labels_np.shape)

In [ ]:
from sklearn.decomposition import PCA
import umap
n_components = 100
n_neighbors = 50
pca = PCA(n_components=n_components)
data_pca = pca.fit_transform(eeg_latent_v)
reducer = umap.UMAP(n_neighbors=n_neighbors, random_state=0)
embedding_2d = reducer.fit_transform(data_pca)


In [ ]:
plt.figure(figsize=(8, 8),dpi=800) 
subject_list = [f'sub-{str(i).zfill(2)}' for i in range(1, 11)]
for label in np.unique(labels):
    indices = labels_np == label
    plt.scatter(embedding_2d[indices, 0][:500], embedding_2d[indices, 1][:500],label=subject_list[label],s=5)

plt.xlabel('UMAP 1', fontsize=22)
plt.ylabel('UMAP 2', fontsize=22)
plt.xticks(fontsize=16) 
plt.yticks(fontsize=16) 
plt.legend(fontsize=15, markerscale=4.0)
plt.tight_layout()
plt.savefig(f'./figures/subject_bias_avg{data_config['data']['test_avg']}.pdf', format='pdf', bbox_inches='tight')

In [ ]:
from scipy.stats import spearmanr,kendalltau,pearsonr
var_list = []

for i in range(10):
    avg = sum(all_list[i]) / len(all_list[i])
    var_list.append(avg)
print(var_list)
top1_acc = [6.1 ,4.9 , 5.6 , 5.0 , 4.0 , 6.0 , 6.5 , 8.8 , 4.3 , 7.0 ]
correlation = np.corrcoef(var_list,top1_acc)[0, 1]
print(correlation)

my_top1_acc = [0.320 ,0.330, 0.355 ,0.390, 0.265, 0.320 ,0.400, 0.480 , 0.350 , 0.415 ] 
my_top1_acc = [100*ele for ele in my_top1_acc]
correlation = np.corrcoef(var_list,my_top1_acc)[0, 1]
print(correlation)


top5_acc = [17.9 , 14.9, 17.4 , 15.1 , 13.4 , 18.2 , 20.4 , 23.7 , 14.0 , 19.7]
correlation = np.corrcoef(var_list,top5_acc)[0, 1]
print(correlation)



def compute_coff(a,b):
    corr_coefficient, p_value = pearsonr(a,b)
    rho, p_value_rho = spearmanr(a,b)
    tau, p_value_tau = kendalltau(a,b)
    return corr_coefficient,rho,tau

top1 = [6.11,4.9,5.58,4.96,4.01,6.01,6.51,8.79,4.34,7.04]
top5 = [17.89,14.87,17.38,15.11,13.39,18.18,20.35,23.68,13.98,19.71]
corr_coefficient,rho,tau = compute_coff(var_list, top1)
print(f"BraVL: top1: {corr_coefficient:.3f},{rho:.3f} {tau:.3f}")
corr_coefficient,rho,tau = compute_coff(var_list, top5)
print(f"BraVL: top5: {corr_coefficient:.3f},{rho:.3f} {tau:.3f}")

top1 = [12.3,10.4,13.1,16.4,8.0 , 14.1,  15.2,  20.0,  13.3,  14.9 ]
top5 = [36.6,33.9,39.0,47.0,26.9, 40.6, 42.1, 49.9, 37.1, 41.9]
corr_coefficient,rho,tau = compute_coff(var_list, top1)
print(f"NICE: top1: {corr_coefficient:.3f},{rho:.3f} {tau:.3f}")
corr_coefficient,rho,tau = compute_coff(var_list, top5)
print(f"NICE: top5: {corr_coefficient:.3f},{rho:.3f} {tau:.3f}")


top1 = [13.3 ,12.1 ,15.3, 15.9 ,9.8,  14.2 , 17.9,18.2 ,14.4 ,16.0] 
top5 = [40.2,36.1 ,39.6 ,49.0 ,34.4 ,42.4 ,43.6 ,50.2,38.7,42.8]
corr_coefficient,rho,tau = compute_coff(var_list, top1)
print(f"NICE-SA: top1: {corr_coefficient:.3f},{rho:.3f} {tau:.3f}")
corr_coefficient,rho,tau = compute_coff(var_list, top5)
print(f"NICE-SA: top5: {corr_coefficient:.3f},{rho:.3f} {tau:.3f}")



top1 = [15.2 , 13.9 , 14.7 , 17.6 , 9.0 , 16.4 , 14.9 , 20.3 , 14.1 , 19.6] 
top5 = [40.1 ,40.1 ,42.7 ,48.9 ,29.7 ,44.4 ,43.1 , 52.1 , 39.7 ,46.7]
corr_coefficient,rho,tau = compute_coff(var_list, top1)
print(f"NICE-GA: top1: {corr_coefficient:.3f},{rho:.3f} {tau:.3f}")
corr_coefficient,rho,tau = compute_coff(var_list, top5)
print(f"NICE-GA: top5: {corr_coefficient:.3f},{rho:.3f} {tau:.3f}")


In [ ]:
from scipy.stats import kendalltau

# 示例数据
x = [1, 2, 3, 4, 5]
y = [4, 1, 3, 5,2]
y = [1, 2, 3, 4, 5]
# 计算 Kendall’s Tau
tau, p_value = kendalltau(x, y)

print("Kendall’s Tau:", tau)
print("p-value:", p_value)
